In [1]:
%autoawait asyncio

In [15]:
import httpx
import asyncio
import aiohttp
import nest_asyncio
import requests

nest_asyncio.apply()

In [11]:
def get_conv_list(host='10.64.136.2:1003'):
    url = f"http://{host}/llm/chat/list"
    payload = ''
    headers = {
        'Content-Type': 'application/json',
        'Lang': 'en'
    }
    resp = requests.request("POST", url, headers=headers, data=payload)
    return resp.json()

In [13]:
conv_list = get_conv_list()

In [31]:
async def gen_code(conv_id, host='10.64.136.2:1003'):
    url = f"http://{host}/llm/gen_code/{conv_id}"
    async with httpx.AsyncClient(timeout=httpx.Timeout(None)) as client:
        data = {"data": "Confirmed, please generate the python protocol script."}
        async with client.stream("POST", url, json=data) as response:
            async for chunk in response.aiter_text():
                pass
    return True

In [32]:
async def handle_request(semaphore, conv_id):
    async with semaphore:
        result = await gen_code(conv_id)
        return result

In [33]:
async def main(conv_list):
    semaphore = asyncio.Semaphore(4)  # Limit to 10 concurrent requests
    async with aiohttp.ClientSession() as session:
        tasks = [handle_request(semaphore, conv['id']) for conv in conv_list]
        results = await asyncio.gather(*tasks)
    return results

In [1]:
# results = asyncio.run(main(cmd))